In [1]:
# **Ceaser Cipher**
def caesar(text, shift):
  result = ""
  for ch in text:
    if ch.islower():
      result += chr((ord(ch) - 97 + shift) % 26 + 97)
    else:
      result += chr((ord(ch) - 65 + shift) % 26 + 65)
  return result
plain  = input()
shift=int(input("enter shift value : "))
enc    = caesar(plain, shift)    # Encrypt with shift = +3
dec    = caesar(enc,  -shift)    # Decrypt by using -3
print("Plain Text :", plain)
print("Encrypted Text :", enc)
print("Decrypted Text :", dec)

Plain Text : saigokul
Encrypted Text : vdljrnxo
Decrypted Text : saigokul


In [2]:
# **Playfair Cipher**

import numpy as np
def generate_key_matrix(key):
    key = "".join(dict.fromkeys(key.upper().replace("J", "I")))
    matrix = list(key + "".join([chr(i) for i in range(65, 91) if chr(i) not in key and chr(i) != "J"]))
    return np.array(matrix).reshape(5, 5)
def find_pos(matrix, char):
    x, y = np.where(matrix == char)
    return x[0], y[0]
def playfair_encrypt(text, matrix):
    text = text.upper().replace("J", "I").replace(" ", "")
    if len(text) % 2 != 0: text += "X"
    result = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        ax,ay=find_pos(matrix,a)
        bx, by = find_pos(matrix, b)
        if ax == bx:
            result += matrix[ax][(ay+1)%5] + matrix[bx][(by+1)%5]
        elif ay == by:
            result += matrix[(ax+1)%5][ay] + matrix[(bx+1)%5][by]
        else:
            result += matrix[ax][by] + matrix[bx][ay]
    return result
def playfair_decrypt(cipher, matrix):
    result = ""
    for i in range(0, len(cipher), 2):
        a, b = cipher[i], cipher[i+1]
        ax, ay = find_pos(matrix, a)
        bx, by = find_pos(matrix, b)
        if ax == bx:
            result += matrix[ax][(ay-1)%5] + matrix[bx][(by-1)%5]
        elif ay == by:
            result += matrix[(ax-1)%5][ay] + matrix[(bx-1)%5][by]
        else:
            result += matrix[ax][by] + matrix[bx][ay]
    return result
matrix = generate_key_matrix("KEYWORD")
cipher = playfair_encrypt("HELLO", matrix)
print("Encrypted:", cipher)
print("Decrypted:", playfair_decrypt(cipher, matrix))

Encrypted: GYFFWZ
Decrypted: HELLOX


In [5]:
# **Hill cipher**

import numpy as np

def hill_encrypt(text, key):
    text = text.upper().replace(" ", "")
    if len(text) % 2 != 0: text += "X"
    result = ""
    for i in range(0, len(text), 2):
        pair = np.array([[ord(text[i]) - 65], [ord(text[i+1]) - 65]])
        enc = np.dot(key, pair) % 26
        result += "".join(chr(x+65) for x in enc.flatten())
    return result

def hill_decrypt(cipher, key):
    det = int(np.round(np.linalg.det(key))) % 26
    inv_det = pow(det, -1, 26)
    key_inv = (inv_det * np.round(det * np.linalg.inv(key)).astype(int)) % 26
    result = ""
    for i in range(0, len(cipher), 2):
        pair = np.array([[ord(cipher[i]) - 65], [ord(cipher[i+1]) - 65]])
        dec = np.dot(key_inv, pair) % 26
        result += "".join(chr(int(x)+65) for x in dec.flatten())
    return result

# Example
key = np.array([[3, 3], [2, 5]])
cipher = hill_encrypt("HELLO", key)
print("Encrypted:", cipher)
print("Decrypted:", hill_decrypt(cipher, key))

Encrypted: HIOZHN
Decrypted: HELLOX


In [3]:
# **Rail fence**

def rail_fence_encrypt(text, rails):
    fence = [["\n"] * len(text) for _ in range(rails)]
    row, down = 0, False
    for i, char in enumerate(text):
        fence[row][i] = char
        if row == 0 or row == rails-1: down = not down
        row += 1 if down else -1
    return "".join("".join(row) for row in fence).replace("\n", "")
def rail_fence_decrypt(cipher, rails):
    fence = [["\n"] * len(cipher) for _ in range(rails)]
    row, down = 0, None
    for i in range(len(cipher)):
        fence[row][i] = "*"
        if row == 0: down = True
        elif row == rails-1: down = False
        row += 1 if down else -1
    idx = 0
    for r in range(rails):
        for c in range(len(cipher)):
            if fence[r][c] == "*" and idx < len(cipher):
                fence[r][c] = cipher[idx]; idx += 1
    result, row, down = "", 0, None
    for i in range(len(cipher)):
        result += fence[row][i]
        if row == 0: down = True
        elif row == rails-1: down = False
        row += 1 if down else -1
    return result
cipher = rail_fence_encrypt("HELLOWORLD", 3)
print("Encrypted:", cipher)
print("Decrypted:", rail_fence_decrypt(cipher, 3))

Encrypted: HOLELWRDLO
Decrypted: HELLOWORLD


In [6]:
# **Row Transposition**

def row_trans_encrypt(plaintext, key, padchar='X'):
    m = len(key)
    rows = []
    for i in range(0, len(plaintext), m):
        chunk = plaintext[i:i+m]   # take a substring of size m
        rows.append(chunk)
    if len(rows[-1]) < m:
        rows[-1] += padchar * (m - len(rows[-1]))
    order = sorted(range(m), key=lambda k: (key[k], k))
    return "".join("".join(row[order[i]] for i in range(m)) for row in rows)

def row_trans_decrypt(ciphertext, key, padchar='X'):
    m = len(key)
    cipher_rows = [ciphertext[i:i+m] for i in range(0, len(ciphertext), m)]
    order = sorted(range(m), key=lambda k: (key[k], k))
    plain = ""
    for crow in cipher_rows:
        row = [''] * m
        for i, ch in enumerate(crow):
            row[order[i]] = ch
        plain += "".join(row)
    return plain.rstrip(padchar)

# example
pt = "HELLOWORLD"
k = "4312"
ct = row_trans_encrypt(pt, k)
print(ct)
print(row_trans_decrypt(ct, k))

LLEHORWOXXDL
HELLOWORLD


In [7]:
# **Column Transposition**

def columnar_trans_encrypt(plaintext, key, padchar='X'):
    m = len(key)
    rows = -(-len(plaintext)//m)
    padded = plaintext + padchar * (rows*m - len(plaintext))
    matrix = [list(padded[i*m:(i+1)*m]) for i in range(rows)]
    order = sorted(range(m), key=lambda k: (key[k], k))
    return "".join(matrix[r][col] for col in order for r in range(rows))

def columnar_trans_decrypt(ciphertext, key, padchar='X'):
    m = len(key)
    rows = -(-len(ciphertext)//m)
    matrix = [['']*m for _ in range(rows)]
    order = sorted(range(m), key=lambda k: (key[k], k))
    idx = 0
    for col in order:
        for r in range(rows):
            matrix[r][col] = ciphertext[idx]; idx += 1
    plain = "".join(''.join(row) for row in matrix)
    return plain.rstrip(padchar)

# example
pt = "HELLOWORLD"
k = "4312"
ct = columnar_trans_encrypt(pt, k)
print(ct)
print(columnar_trans_decrypt(ct, k))

LOXLRXEWDHOL
HELLOWORLD


In [8]:
##DES
!pip install pycryptodome
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad

key = b"8bytekey"
plaintext = b"THISISLABFOUR"
cipher = DES.new(key, DES.MODE_ECB)
ct = cipher.encrypt(pad(plaintext, DES.block_size))
pt = unpad(cipher.decrypt(ct), DES.block_size)

print("Encrypted:", ct)
print("Decrypted:", pt)

Defaulting to user installation because normal site-packages is not writeable
Encrypted: b'==q\n\xbd\x9a\x99\xef"\x90kCs\x9fh\xd9'
Decrypted: b'THISISLABFOUR'



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# **3-DES**

from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes

key = DES3.adjust_key_parity(get_random_bytes(24))
cipher = DES3.new(key, DES3.MODE_EAX)

plaintext = b"Hello 3DES"
ct, tag = cipher.encrypt_and_digest(plaintext)
nonce = cipher.nonce

decipher = DES3.new(key, DES3.MODE_EAX, nonce=nonce)
pt = decipher.decrypt(ct)

print("Encrypted:", ct)
print("Decrypted:", pt)

Encrypted: b'\xf34,\x9eb\xae\xba\xa6ql'
Decrypted: b'Hello 3DES'


In [10]:
# **AES**

!pip install pycryptodome
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)  # 128-bit
cipher = AES.new(key, AES.MODE_EAX)

plaintext = b"Hello AES"
ct,_= cipher.encrypt_and_digest(plaintext)
nonce = cipher.nonce

decipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
pt = decipher.decrypt(ct)

print("Encrypted:", ct)
print("Decrypted:", pt)

Defaulting to user installation because normal site-packages is not writeable
Encrypted: b'Dyp\xbf\xacd|\xee\xd8'
Decrypted: b'Hello AES'



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# **RSA**

p = 7919
q = 1009
n = p * q
phi = (p-1) * (q-1)
e = 5
def mod_inverse(a, m):
    for d in range(1, m):
        if (a * d) % m == 1:
            return d
    return None
d = mod_inverse(e, phi)
print("Public Key (e, n):", (e, n))
print("Private Key (d, n):", (d, n))
def encrypt(message, e, n):
    return [(ord(ch) ** e) % n for ch in message]
def decrypt(cipher, d, n):
    return "".join([chr((c ** d) % n) for c in cipher])
msg = "Ram"
cipher = encrypt(msg, e, n)
plain = decrypt(cipher, d, n)
print("Original Message:", msg)
print("Encrypted:", cipher)
print("Decrypted:", plain)

Public Key (e, n): (5, 7990271)
Private Key (d, n): (1596269, 7990271)
Original Message: Ram
Encrypted: [7902959, 5789203, 4967874]
Decrypted: Ram


In [12]:
# Diffie-Hellman Key Exchange
def power_mod(base, exponent, modulus):
    return pow(base, exponent, modulus)

p = 23  # prime number
g = 5   # generator

a = 6   # Alice's private key
b = 15  # Bob's private key

A = power_mod(g, a, p)  # Alice's public key
B = power_mod(g, b, p)  # Bob's public key

shared_secret_alice = power_mod(B, a, p)
shared_secret_bob = power_mod(A, b, p)

print("Shared Secret (Alice):", shared_secret_alice)
print("Shared Secret (Bob):", shared_secret_bob)

Shared Secret (Alice): 2
Shared Secret (Bob): 2


In [14]:
##SHA-256 Hash Function
import hashlib

def sha256_hash(text):
    # Encode string to bytes
    encoded_text = text.encode()
    # Create SHA-256 hash object
    hash_object = hashlib.sha256(encoded_text)
    # Return hexadecimal representation
    return hash_object.hexdigest()

# Example usage
data = "Hello, World!"
print("SHA-256:", sha256_hash(data))

SHA-256: dffd6021bb2bd5b0af676290809ec3a53191dd81c7f70a4b28688a362182986f


WEEK 10 – Digital Signature Standard (DSS/DSA)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes

def generate_keys():
    private_key = dsa.generate_private_key(key_size=1024)
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    return private_key.sign(message, hashes.SHA256())

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(signature, message, hashes.SHA256())
        return True
    except Exception:
        return False

message = b"Hello, this message will be signed."

priv, pub = generate_keys()
signature = sign_message(priv, message)

print("Signature (Hex):", signature.hex())
print("Verification:", verify_signature(pub, message, signature))

WEEK 11 – SSL and Intrusion Detection System (IDS)

Server Code

In [ ]:
import socket, ssl

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
ssl_context.load_cert_chain(certfile="server.crt", keyfile="server.key")

server_socket.bind(('localhost', 8443))
server_socket.listen(5)
print("SSL Server running...")

with ssl_context.wrap_socket(server_socket, server_side=True) as ssock:
    conn, addr = ssock.accept()
    data = conn.recv(1024).decode()
    print("Received:", data)
    conn.sendall(b"Hello from SSL Server!")
    conn.close()

Client Code

In [ ]:
import socket, ssl

hostname = 'localhost'

# Create an SSL context that ignores certificate verification (for demo)
context = ssl._create_unverified_context()

with socket.create_connection((hostname, 8443)) as sock:
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        print("✅ SSL connection established.")
        ssock.sendall(b"Hello from SSL Client!")
        data = ssock.recv(1024).decode()
        print("Received:", data)

generate_cert.py

In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from datetime import datetime, timedelta

# Generate private key
key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Write private key to file
with open("server.key", "wb") as f:
    f.write(key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Build self-signed certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "IN"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Telangana"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Hyderabad"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "SSL Demo"),
    x509.NameAttribute(NameOID.COMMON_NAME, "localhost"),
])

cert = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(issuer)
    .public_key(key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.utcnow())
    .not_valid_after(datetime.utcnow() + timedelta(days=365))
    .sign(key, hashes.SHA256())
)

# Write certificate to file
with open("server.crt", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

print("✅ Generated server.crt and server.key successfully!")